# Test decoding of confocal MERFISH

by Pu Zheng

2025.7.25

In [1]:
!ls /lab/weissman_imaging/puzheng/Softwares/

'Batch Denoise.ga3'	      Microscope
 Baysor			      miniconda3
 Cassiopeia		      nupack-4.0.1.12
 ChromAn		      nupack-4.0.1.12.zip
 Chromatin_Analysis_Scripts   PEtracer-2025
 Confocal		      PETracer_Paper
 cpsam			      postanalysis.yml
 deconwolf		      pycea
 deconwolf_old		      RDiT
 fishtank		      resolVI
 fishtank_environment.yml     segger
 ImageAnalysis3		      segment-anything
 Merfish_Analysis_Scripts     Sequencing
 MERFISH_probe_design	      ultima_spatial
 MERlin			      Weissman_MERFISH_Scripts


In [2]:
# import required packages:
import os,sys,time,h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import local packages:
sys.path.append('/lab/weissman_imaging/puzheng/Softwares/')

In [18]:
import ChromAn
from ChromAn.src.file_io.data_organization import search_fovs_in_folders, Color_Usage
from ChromAn.src.file_io.dax_process import DaxProcesser

In [8]:
# define datapath:
data_folder = r'/lab/weissman_imaging/puzheng/4T1Tumor/20250331-4T1FE_MF9-4pp'
folders, fovs = search_fovs_in_folders(data_folder,)

- searching in folder: /lab/weissman_imaging/puzheng/4T1Tumor/20250331-4T1FE_MF9-4pp
-- 10 folders, 41 fovs detected.


In [35]:
os.listdir(data_folder+r'/Analysis')

['positions.txt', 'color_usage_MF9-4plusplus.csv']

In [16]:
# Load color_usage:
color_usage_filename = os.path.join(data_folder, 'Analysis', 'color_usage_MF9-4plusplus.csv')
color_usage = Color_Usage(color_usage_filename)

- load color_usage from file: /lab/weissman_imaging/puzheng/4T1Tumor/20250331-4T1FE_MF9-4pp/Analysis/color_usage_MF9-4plusplus.csv


In [17]:
color_usage

,748,637,477,405
Hyb,,,,
H0M1,m1,m2,beads,DAPI
H1M2,m3,m4,beads,NaN
H2M3,m5,m6,beads,NaN
H3M4,m7,m8,beads,NaN
H4M5,m9,m10,beads,NaN
H5M6,m11,m12,beads,NaN
H6M7,m13,m14,beads,NaN
H7M8,m15,m16,beads,NaN
H8M9,m17,m18,beads,NaN


## Now, load all raw-images

In [28]:
sel_fov = fovs[15]
hyb_2_daxp = {}
correction_folder = r'/lab/weissman_imaging/puzheng/Corrections/20240401-Merscope01_s11_n1200'
fiducial_channel = color_usage.get_fiducial_channel(color_usage)

for _hyb, _info in color_usage.iterrows():
    # load daxp:
    daxp_filename = os.path.join(data_folder, _hyb, sel_fov)
    hyb_2_daxp[_hyb] = DaxProcesser(daxp_filename, 
                                    FiducialChannel=fiducial_channel,
                                    CorrectionFolder=correction_folder,
                                    )
# load images:


Initialize DaxProcesser for file:/lab/weissman_imaging/puzheng/4T1Tumor/20250331-4T1FE_MF9-4pp/H0M1/Conv_zscan_15.dax
- New save file: /lab/weissman_imaging/puzheng/4T1Tumor/20250331-4T1FE_MF9-4pp/H0M1/Conv_zscan_15_processed.hdf5
-- all used channels: ['748', '637', '477', '405']
Initialize DaxProcesser for file:/lab/weissman_imaging/puzheng/4T1Tumor/20250331-4T1FE_MF9-4pp/H1M2/Conv_zscan_15.dax
- New save file: /lab/weissman_imaging/puzheng/4T1Tumor/20250331-4T1FE_MF9-4pp/H1M2/Conv_zscan_15_processed.hdf5
-- all used channels: ['748', '637', '477']
Initialize DaxProcesser for file:/lab/weissman_imaging/puzheng/4T1Tumor/20250331-4T1FE_MF9-4pp/H2M3/Conv_zscan_15.dax
- New save file: /lab/weissman_imaging/puzheng/4T1Tumor/20250331-4T1FE_MF9-4pp/H2M3/Conv_zscan_15_processed.hdf5
-- all used channels: ['748', '637', '477']
Initialize DaxProcesser for file:/lab/weissman_imaging/puzheng/4T1Tumor/20250331-4T1FE_MF9-4pp/H3M4/Conv_zscan_15.dax
- New save file: /lab/weissman_imaging/puzheng/4T1

In [29]:
# load images:
for _hyb, _daxp in hyb_2_daxp.items():
    _daxp._load_image()
    if _hyb != 'H0M1':
        _daxp._calculate_drift(getattr(hyb_2_daxp['H0M1'], f'im_{fiducial_channel}'))       

-- single image size: [  11 2304 2304]
- Loaded images for channels:['748', '637', '477', '405'] in 3.227s.
-- single image size: [  11 2304 2304]
- Loaded images for channels:['748', '637', '477'] in 2.641s.
+ Calculate drift with fiducial_channel: 477
-- start aligning given source image to given reference image.
-- drift 0: [-0.01 -3.04  1.5 ] in 0.637s.
-- drift 1: [ 0.   -3.1   1.44] in 0.592s.
-- drift 2: [ 0.   -3.08  1.54] in 0.601s.
--- drifts for crops:[0 1 2] pass the thresold, exit cycle.
-- single image size: [  11 2304 2304]
- Loaded images for channels:['748', '637', '477'] in 3.288s.
+ Calculate drift with fiducial_channel: 477
-- start aligning given source image to given reference image.
-- drift 0: [ 0.   -2.28  0.28] in 0.611s.
-- drift 1: [ 0.   -2.33  0.28] in 0.603s.
-- drift 2: [ 0.   -2.33  0.19] in 0.598s.
--- drifts for crops:[0 1 2] pass the thresold, exit cycle.
-- single image size: [  11 2304 2304]
- Loaded images for channels:['748', '637', '477'] in 2.2

In [34]:
for _hyb, _daxp in hyb_2_daxp.items():
    if hasattr(_daxp, 'drift'):
        print(f'{_hyb} has drift: {_daxp.drift}')

H1M2 has drift: [-0.00333333 -3.07333333  1.49333333]
H2M3 has drift: [ 0.         -2.31333333  0.25      ]
H3M4 has drift: [ 0.         -1.67333333 -0.14333333]
H4M5 has drift: [ 0.         -1.37666667 -1.55333333]
H5M6 has drift: [ 0.         -1.53333333 -2.16666667]
H6M7 has drift: [ 0.         -1.50666667 -2.73333333]
H7M8 has drift: [ 0.01       -0.29666667 -3.18      ]
H8M9 has drift: [ 0.01        0.87333333 -4.41      ]
H9M10 has drift: [  0.03         2.60666667 -10.62333333]


In [ ]:
# now, run the corrections:
for _hyb, _daxp in hyb_2_daxp.items():
    if hasattr(_daxp, 'drift'):
        #print(f'{_hyb} has drift: {_daxp.drift}')